In [1]:
import os
os.environ["SPARK_HOME"] = "D:\RISET\Spark\spark-3.1.1-bin-hadoop2.7"
from platform import python_version
print(python_version())

3.7.10


In [2]:
import numpy as np
import pandas as pd
import xlwings as xw
import folium
import docplex.mp.solution as mp_sol
from docplex.mp.model import Model
from geopy.distance import great_circle
import json

n = 50
customer = [i for i in range(1, n + 1)]
customer2 = [i for i in range(1, n + 1)]
node = [0] + customer
arcos = [(i,j) for i in node for j in node if i != j]
f = open('pelanggan4.json')
data = json.load(f)

# get customer demand and location data
df = pd.DataFrame(data)

# get distance & duration data
distance = np.loadtxt('distance.txt')
duration = np.loadtxt('duration.txt')
vehicle_capacity = 250

# Start time
e = [df['ready_time'][i] for i in range(n + 1)]
e[0] = 0

# Due time
l = [df['due_time'][i] for i in range(n + 1)]
 

# Service time
ser = [df['service_time'][i] for i in range(n + 1)]
ser[0] = 0

M = 5000  # big number
no_of_vehicles = 3  # free to modify
n = n + 1

In [3]:
mdl = Model('CVRPTW')
x = mdl.binary_var_dict(arcos, name='x')
u = mdl.continuous_var_dict(customer, name='u')
t = mdl.continuous_var_dict(customer2, name='t')

In [4]:
mdl.add_constraints(mdl.sum(x[(i,j)] for i,j in arcos if i==h)== 1 for h in customer)# vehicles leaving each customer node
mdl.add_constraints(mdl.sum(x[(i,j)] for i,j in arcos if j==h)== 1 for h in customer)# vehicles arriving to each customer node
# mdl.add_constraint(mdl.sum(x[(0,j)] for j in customer) == no_of_vehicles)# vehicles leaving depot
# mdl.add_constraint(mdl.sum(x[(i,0)] for i in customer) == no_of_vehicles)# vehicles arriving to depot

[docplex.mp.LinearConstraint[](x_0_1+x_2_1+x_3_1+x_4_1+x_5_1+x_6_1+x_7_1+x_8_1+x_9_1+x_10_1+x_11_1+x_12_1+x_13_1+x_14_1+x_15_1+x_16_1+x_17_1+x_18_1+x_19_1+x_20_1+x_21_1+x_22_1+x_23_1+x_24_1+x_25_1+x_26_1+x_27_1+x_28_1+x_29_1+x_30_1+x_31_1+x_32_1+x_33_1+x_34_1+x_35_1+x_36_1+x_37_1+x_38_1+x_39_1+x_40_1+x_41_1+x_42_1+x_43_1+x_44_1+x_45_1+x_46_1+x_47_1+x_48_1+x_49_1+x_50_1,EQ,1),
 docplex.mp.LinearConstraint[](x_0_2+x_1_2+x_3_2+x_4_2+x_5_2+x_6_2+x_7_2+x_8_2+x_9_2+x_10_2+x_11_2+x_12_2+x_13_2+x_14_2+x_15_2+x_16_2+x_17_2+x_18_2+x_19_2+x_20_2+x_21_2+x_22_2+x_23_2+x_24_2+x_25_2+x_26_2+x_27_2+x_28_2+x_29_2+x_30_2+x_31_2+x_32_2+x_33_2+x_34_2+x_35_2+x_36_2+x_37_2+x_38_2+x_39_2+x_40_2+x_41_2+x_42_2+x_43_2+x_44_2+x_45_2+x_46_2+x_47_2+x_48_2+x_49_2+x_50_2,EQ,1),
 docplex.mp.LinearConstraint[](x_0_3+x_1_3+x_2_3+x_4_3+x_5_3+x_6_3+x_7_3+x_8_3+x_9_3+x_10_3+x_11_3+x_12_3+x_13_3+x_14_3+x_15_3+x_16_3+x_17_3+x_18_3+x_19_3+x_20_3+x_21_3+x_22_3+x_23_3+x_24_3+x_25_3+x_26_3+x_27_3+x_28_3+x_29_3+x_30_3+x_31_3+x_3

In [5]:
'''constraint_6: time-windows and also eliminating sub-tours'''

for i in customer:
    # assumption: service starts at 9:00 AM, 9 == 0 minutes, each hour after 9 is 60 minutes plus previous hours
    mdl.add_constraint(t[i] >= e[i])  # service should start after the earliest service start time
    mdl.add_constraint(t[i] <= l[i])  # service can't be started after the latest service start time
    for j in customer:
        if i!=j:
            # taking the linear distance from one node to other as travelling time in minutes between those nodes
            mdl.add_indicator(x[(i,j)], t[i] + ser[i] + duration[i][j] <= t[j])

In [6]:
'''constraint_5: capacity of vehicle and also eliminating sub-tours'''
for j in customer:
    mdl.add_constraint(u[j] <= vehicle_capacity)
    mdl.add_constraint(u[j] >= df.demand[j])
    for i in customer:
        if i!= j:
            mdl.add_constraint(u[j] >= u[i] + df.demand[j] * (x[i,j]) - vehicle_capacity * (1 - (x[i,j])))

In [7]:
'''objective function'''
mdl.minimize(mdl.sum(distance[(i, j)] * x[(i, j)] for i, j in arcos))

In [8]:
mdl.parameters.timelimit = 60
# mdl.parameters.parallel = 1
# mdl.parameters.threads =  0
# mdl.parameters.benders.strategy = -1
# mdl.parameters.mip.limits.cutsfactor = 0
# mdl.parameters.mip.tolerances.mipgap = 0.1
# mdl.parameters.mip.strategy.branch = 0

###Menghitung Waktu
import time
start_timer = time.time()

solucion = mdl.solve(log_output=True)

###Menghitung Waktu
done_timer = time.time()
elapsed = done_timer - start_timer

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               60
Found incumbent of value 346412.000000 after 0.00 sec. (0.30 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 748 rows and 548 columns.
MIP Presolve modified 4488 coefficients.
Aggregator did 684 substitutions.
Reduced MIP has 5122 rows, 3320 columns, and 17068 nonzeros.
Reduced MIP has 2002 binaries, 0 generals, 0 SOSs, and 1902 indicators.
Presolve time = 0.05 sec. (20.87 ticks)
Probing time = 0.13 sec. (15.91 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 5122 rows, 3320 columns, and 17068 nonzeros.
Reduced MIP has 2002 binaries, 0 generals, 0 SOSs, and 1902 indicators.
Presolve time = 0.03 sec. (10.91 ticks)
Probing time = 0.03 sec. (5.19 ticks)
Clique table members: 10736.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using 

   1667     0    70536.9444   147   125016.0000     Cuts: 310   210298   43.58%
   1667     0    70541.5884   147   125016.0000     Cuts: 279   210394   43.57%
   1667     0    70552.5103   147   125016.0000      Cuts: 26   210510   43.57%
   1667     0    70576.6442   147   125016.0000     Cuts: 911   210736   43.55%
   1667     0    70588.0592   147   125016.0000     Cuts: 390   210853   43.54%
   1667     0    70620.9720   147   125016.0000     Cuts: 446   210967   43.51%
   1667     0    70627.3865   147   125016.0000     Cuts: 140   211126   43.51%
   1667     0    70651.8669   147   125016.0000      Cuts: 18   211292   43.49%
   1667     0    70716.6177   147   125016.0000     Cuts: 891   211755   43.43%
   1667     0    70749.3457   147   125016.0000     Cuts: 763   211940   43.41%
   1667     0    70752.5135   147   125016.0000     Cuts: 281   212006   43.41%
   1667     0    70756.0773   147   125016.0000      Cuts: 47   212045   43.40%
Starting limited solution polishing.
   

In [9]:
mdl.get_solve_status()

<JobSolveStatus.FEASIBLE_SOLUTION: 1>

In [10]:
# Encontrar las rutas por separado
route = []
total_dis = []
for h in customer:
    if x[(0, h)].solution_value > 0.9:
        arcos_route = [(0, h)]
        item_dis = distance[0][h]
        j = h
        while j!=0:
            i = j
            for k in node:
                if i!=k and x[(i, k)].solution_value > 0.9:
                    j = k
                    arcos_route.append((i, j))
                    item_dis+= distance[i][j]
                    break
        route.append(arcos_route)
        print(arcos_route)
        print("----------------------")
        total_dis.append(item_dis)


[(0, 10), (10, 1), (1, 9), (9, 8), (8, 20), (20, 0)]
----------------------
[(0, 11), (11, 2), (2, 24), (24, 0)]
----------------------
[(0, 12), (12, 7), (7, 13), (13, 32), (32, 6), (6, 26), (26, 21), (21, 0)]
----------------------
[(0, 18), (18, 19), (19, 4), (4, 5), (5, 15), (15, 14), (14, 0)]
----------------------
[(0, 28), (28, 50), (50, 33), (33, 41), (41, 25), (25, 0)]
----------------------
[(0, 29), (29, 43), (43, 48), (48, 0)]
----------------------
[(0, 34), (34, 23), (23, 27), (27, 22), (22, 0)]
----------------------
[(0, 35), (35, 46), (46, 30), (30, 31), (31, 0)]
----------------------
[(0, 36), (36, 42), (42, 45), (45, 0)]
----------------------
[(0, 40), (40, 39), (39, 37), (37, 38), (38, 0)]
----------------------
[(0, 44), (44, 47), (47, 0)]
----------------------
[(0, 49), (49, 16), (16, 17), (17, 3), (3, 0)]
----------------------


In [11]:
best_distance = 0
for n in range(len(route)):
    print("Rute "+str(n))
    print(str(route[n])+" -> "+str(total_dis[n]))
    best_distance+=total_dis[n]
print("Total Distace : "+str(best_distance))
print(elapsed)

Rute 0
[(0, 10), (10, 1), (1, 9), (9, 8), (8, 20), (20, 0)] -> 8766.0
Rute 1
[(0, 11), (11, 2), (2, 24), (24, 0)] -> 5446.0
Rute 2
[(0, 12), (12, 7), (7, 13), (13, 32), (32, 6), (6, 26), (26, 21), (21, 0)] -> 8456.0
Rute 3
[(0, 18), (18, 19), (19, 4), (4, 5), (5, 15), (15, 14), (14, 0)] -> 10127.0
Rute 4
[(0, 28), (28, 50), (50, 33), (33, 41), (41, 25), (25, 0)] -> 13664.0
Rute 5
[(0, 29), (29, 43), (43, 48), (48, 0)] -> 10246.0
Rute 6
[(0, 34), (34, 23), (23, 27), (27, 22), (22, 0)] -> 6697.0
Rute 7
[(0, 35), (35, 46), (46, 30), (30, 31), (31, 0)] -> 10725.0
Rute 8
[(0, 36), (36, 42), (42, 45), (45, 0)] -> 16011.0
Rute 9
[(0, 40), (40, 39), (39, 37), (37, 38), (38, 0)] -> 9600.0
Rute 10
[(0, 44), (44, 47), (47, 0)] -> 13221.0
Rute 11
[(0, 49), (49, 16), (16, 17), (17, 3), (3, 0)] -> 11731.0
Total Distace : 124690.0
60.080748558044434


In [12]:
best_distance = 0
for n in range(len(route)):
    print("Rute "+str(n))
    print(str(route[n])+" -> "+str(total_dis[n]))
    s_time = 0
    demands = 0
    for rt in route[n]:
        (i,j) = rt
        demands+=df.demand[j]
        s_time += duration[i][j]
        print(str(rt)+" --> "+str(e[j])+" - "+str(l[j])+" --> "+str(s_time))
        s_time += ser[j]
    print("Total Demands : "+str(demands))
    print("----------")
    best_distance+=total_dis[n]
print("Total Distace : "+str(best_distance))

print(elapsed)

Rute 0
[(0, 10), (10, 1), (1, 9), (9, 8), (8, 20), (20, 0)] -> 8766.0
(0, 10) --> 117 - 1317 --> 266.0
(10, 1) --> 987 - 2187 --> 661.0
(1, 9) --> 605 - 1805 --> 959.0
(9, 8) --> 860 - 2060 --> 1257.0
(8, 20) --> 914 - 2114 --> 1652.0
(20, 0) --> 0 - 5000 --> 2569.0
Total Demands : 208
----------
Rute 1
[(0, 11), (11, 2), (2, 24), (24, 0)] -> 5446.0
(0, 11) --> 1078 - 2278 --> 290.0
(11, 2) --> 367 - 1567 --> 755.0
(2, 24) --> 1151 - 2351 --> 1033.0
(24, 0) --> 0 - 5000 --> 1571.0
Total Demands : 133
----------
Rute 2
[(0, 12), (12, 7), (7, 13), (13, 32), (32, 6), (6, 26), (26, 21), (21, 0)] -> 8456.0
(0, 12) --> 140 - 1340 --> 377.0
(12, 7) --> 219 - 1419 --> 771.0
(7, 13) --> 687 - 1887 --> 1013.0
(13, 32) --> 151 - 1351 --> 1334.0
(32, 6) --> 810 - 2010 --> 1697.0
(6, 26) --> 958 - 2158 --> 2197.0
(26, 21) --> 1173 - 2373 --> 2588.0
(21, 0) --> 0 - 5000 --> 2945.0
Total Demands : 211
----------
Rute 3
[(0, 18), (18, 19), (19, 4), (4, 5), (5, 15), (15, 14), (14, 0)] -> 10127.0
(0, 18